In [1]:
import torch
import os
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow.keras.models import load_model
import requests
from sklearn.metrics.pairwise import cosine_similarity
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM, AutoModel

In [2]:
load_dotenv()

True

In [3]:
model = load_model('LSTM_euler_temporal_dynamics')

In [4]:
sample_input_sequence=np.array([[-9.28317111e-01, -9.26705018e-01, -1.33821092e+00,
         1.59151996e-01, -9.80534562e-01,  1.13180968e+00,
        -8.14734264e-01, -1.29092260e-02, -1.01997518e+00,
         2.10817179e+00,  2.12091685e-02, -1.51470508e-03,
        -4.27905404e-02,  1.15207013e+00,  1.26888526e+00,
        -6.85882976e-01, -1.34530801e-01, -2.87846841e-02,
        -8.30967007e-01,  2.60652480e+00, -2.04741732e-01,
        -1.45619779e-03]])

In [5]:
sample_sequence = np.expand_dims(sample_input_sequence, axis=1)

In [6]:
predictions = model.predict(sample_sequence)

1/1 [==============================] - 0s 354ms/step


In [7]:
categorical_features = ['armraise', 'tricepkickback', 'overheadpress', 'bicepcurl',
                        'hammercurl', 'burpee', 'vup', 'crunch', 'pushup', 'birddog'] 
categories_array = np.array(categorical_features).reshape(-1, 1)
encoder = OneHotEncoder(sparse_output=False)
target_encoded = encoder.fit_transform(categories_array)

In [8]:
predicted_indices = np.argmax(predictions, axis=1)
predicted_one_hot = np.zeros((predicted_indices.size, predictions.shape[1]))
predicted_one_hot[np.arange(predicted_indices.size), predicted_indices] = 1

predicted_labels = encoder.inverse_transform(predicted_one_hot)

In [9]:
predicted_user_exercise=predicted_labels[0][0]

RAG setup

In [10]:
df = pd.read_csv('megaGymDataset.csv')

In [11]:
df.head()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [12]:
df.isnull().sum() 

Unnamed: 0       0
Title            0
Desc          1550
Type             0
BodyPart         0
Equipment        0
Level            0
Rating        1887
RatingDesc    2056
dtype: int64

In [13]:
df = df.dropna(subset=['Desc'])

In [14]:
df ['exercise_description'] = df ['Desc'] + ' It is a ' + df ['Type'] + ' exercise.' + ' It focusses on ' + df['BodyPart'] + ' body part.' + 'It is an '+ df['Level'] + ' exercise.'+ ' To perform this exercise, one must use '+ df['Equipment']

In [15]:
dataset_exercise_description={"armraise":"The exercise involves standing upright and lifting the arms outward to the side until they are at shoulder level, then lowering them back down.",
    "tricepkickback": "Performed by leaning forward slightly with a dumbbell in each hand, elbows bent, and then extending the arms to push the weights back and up.",
    "overheadpress":"A strength-building exercise that involves pressing weights upward from shoulder height until the arms are extended overhead.",
    "bicepcurl": "TThis exercise is done by holding weights in both hands and curling the arms upward while keeping the elbows close to the torso.",
    "hammercurl" : "Similar to the bicep curl but performed with the palms facing each other. It engages both the biceps and the forearm muscles.",
    "burpee":"A full-body exercise that combines a squat, jump, and pushup. It is known for its effectiveness in building strength and endurance.",
    "vup" : "An abdominal exercise where one simultaneously lifts the legs and upper body off the ground, forming a 'V' shape with the body.",
    "crunch": "Performed by lying on the back with knees bent, then curling the shoulders towards the pelvis.",
    "pushup": "A fundamental body-weight exercise that involves lowering and raising the body using the arms while keeping the body straight.",
    "birddog": "A core strengthening exercise that also improves balance and stability. It involves extending one arm and the opposite leg while kneeling on all fours."}

In [16]:
df= df.drop(['Unnamed: 0','Desc','Type','BodyPart','Level','Equipment','Rating','RatingDesc'],axis=1)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [18]:
def encode_text(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings

df['embeddings'] = df['Title'].apply(lambda x: encode_text([x]).numpy())
df['embeddings'].to_pickle('embeddings.pkl')
print("Embeddings generated and saved.")

Embeddings generated and saved.


In [19]:
def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

In [20]:
def calculate_similarities(embeddings, user_embedding):
    dataset_embeddings = np.array(embeddings)
    if dataset_embeddings.ndim == 3 and dataset_embeddings.shape[1] == 1:
        dataset_embeddings = dataset_embeddings.reshape(dataset_embeddings.shape[0], dataset_embeddings.shape[2])

    if user_embedding.ndim == 2:
        user_embedding = user_embedding.reshape(1, -1)  # Ensuring it is two-dimensional

    return cosine_similarity(dataset_embeddings, user_embedding).flatten()

LLM based Tailored suggestions for Predictions

In [21]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [22]:
User_profile = "Elena Martinez, a 42-year-old graphic designer, struggles with chronic tennis elbow, resulting in pain on the outside of her elbow and weakened grip strength. Her condition, exacerbated by the repetitive strain of her profession, is managed through physical therapy and corticosteroid injections. Elena's primary fitness goal is to strengthen her arm muscles without worsening her symptoms, while also aiming to enhance her overall upper body strength and joint flexibility. She faces significant challenges, including the need to avoid high-impact and repetitive arm movements, and she often grapples with frustration over her slow progress and anxiety about potential re-injuries. Her daily routine involves prolonged computer work, interspersed with light exercises and bi-weekly physical therapy sessions."

In [23]:
predicted_exercise_embedding = generate_embedding(predicted_user_exercise)

In [24]:
similarities_1 = calculate_similarities(df['embeddings'].tolist(), predicted_exercise_embedding)
df['similarity'] = similarities_1
df_sorted = df.sort_values(by='similarity', ascending=False)
print(df_sorted[['Title', 'similarity','exercise_description']].head(3))

grounded_exercise_description = df_sorted['exercise_description'].iloc[0]
print(grounded_exercise_description)

                      Title  similarity  \
390               Bird dog-    0.848398   
254           Half bird dog    0.743157   
649  Exercise ball bird dog    0.626746   

                                  exercise_description  
390  The bird dog is a core exercise designed to he...  
254  The half bird dog is a core exercise designed ...  
649  The exercise ball bird dog is a dynamic moveme...  
The bird dog is a core exercise designed to help strengthen and stabilize the posterior chain. It involves lifting an opposite-side arm and leg while in a bent-leg tabletop position. It is most commonly performed as part of a dynamic warm-up, or for mobility and muscle activation anytime. It is a Strength exercise. It focusses on Abdominals body part.It is an Intermediate exercise. To perform this exercise, one must use Body Only


In [25]:
user_message=f"The user profile : {User_profile}, the exercise the user is performing: {predicted_user_exercise} and the exercise description is {grounded_exercise_description}"

Doctor Persona

In [26]:
response_1 = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "As an orthopedic specialist, your role involves advising individuals on how to maintain a healthy musculoskeletal system through appropriate physical activities. Given a user's profile and the specific exercise they are performing, analyze the typical physical movements associated with this exercise. Based on your knowledge, provide detailed suggestions and insights on how to perform these exercises correctly to prevent injuries and optimize musculoskeletal health."},
    {"role": "user", "content": user_message}
  ]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
message = response_1.choices[0].message.content

In [28]:
message

"Elena, the bird dog exercise can be a beneficial addition to your routine, particularly for improving core stability and overall body strength without placing excessive strain on your arm or elbow. Let’s break down the movement and provide you some insights on how to perform it correctly, with special attention to managing your elbow condition.\n\n### Overview of the Bird Dog Exercise:\n\nThe bird dog works to engage and strengthen the core, glutes, and lower back, which are essential for overall body stability. This will also help in distributing loads more evenly throughout your body, potentially alleviating some strain on your upper body.\n\n### Proper Technique for Performing the Bird Dog:\n\n1. **Starting Position:**\n   - Begin in a tabletop position with your hands directly under your shoulders and knees under your hips. \n   - Ensure your back is flat, maintaining a neutral spine (not rounding or arching).\n\n2. **Engage Your Core:**\n   - Before initiating movement, draw your

In [29]:
user_message = f"""Analyze the given exercise using the information provided below. Identify key movements and techniques that ensure safe execution and minimize injury risk. Offer insights into how the exercise affects the joints and muscles, and provide recommendations for optimal form and technique. Discuss potential modifications for individuals with specific conditions or limitations, and explain how this exercise can contribute to long-term joint stability and musculoskeletal health.

- User Profile: {User_profile}
- Exercise Being Done: {predicted_user_exercise}
- Grounded Description of the Exercise: {grounded_exercise_description}"""

In [30]:
response_2 = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are an orthopedic specialist. Your role is to provide a detailed analysis and insights on the execution of exercises to support musculoskeletal health."},
      {"role":"user", "content": user_message}
     ]
)

In [31]:
message = response_2.choices[0].message.content

In [32]:
message

"### Analysis of the Bird Dog Exercise for Elena Martinez\n\nThe bird dog exercise is an excellent choice for Elena, given her background and specific musculoskeletal concerns. Below is a detailed breakdown of the key movements, techniques for safe execution, joint and muscle effects, optimal form, modifications for her condition, and the contribution of this exercise to long-term joint stability and musculoskeletal health.\n\n#### Key Movements and Techniques for Safe Execution\n\n1. **Starting Position**: \n   - Start on all fours in a tabletop position. \n   - Ensure that knees are beneath the hip joints and the hands are beneath the shoulders.\n   - The spine should be in a neutral position, avoiding excessive arching or rounding.\n\n2. **Movement Execution**:\n   - **Extension**: Extend the right arm forward and the left leg backward, focusing on maintaining a stable core. \n   - **Alignment**: Keep the hips square to the ground and avoid twisting; the limbs should form a straight

Fitness Coaches

In [33]:
user_message=f"""Analyze the given exercise using the information provided below. Provide metrics and strategies for improving the user's performance. Discuss how the exercise aligns with the user's fitness goals and offer tips for optimizing technique and maximizing benefits. Include advice on progression, such as increasing resistance or intensity, and emphasize performance metrics that indicate improvement, such as form precision, repetitions, and endurance. Provide motivational tips to keep the user engaged and progressing towards their fitness objectives.

- User Profile: {User_profile}
- Exercise Being Done: {predicted_user_exercise}
- Grounded Description of the Exercise: {grounded_exercise_description}
"""

In [34]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a fitness coach. Your goal is to enhance exercise performance and effectiveness."},
    {"role": "user", "content": user_message}
  ]
)

In [35]:
message = response.choices[0].message.content

In [36]:
message

"### Exercise Analysis: Bird Dog\n\n**Overview**:\nThe bird dog exercise is particularly beneficial for core stability, spinal alignment, and overall functional strength, all crucial for someone like Elena, who is managing chronic tennis elbow and looking to enhance her upper body strength without causing additional strain. \n\n### Performance Metrics\n\n1. **Form Precision**: Ensure that Elena maintains a neutral spine and that the movements with the arm and leg are controlled. The hips should remain square to the ground.\n2. **Repetitions**: Start with sets of 5-8 repetitions on each side, aiming for 2-3 sets. As her comfort with the exercise increases and her pain decreases, she can gradually increase the number of reps, working toward 10-15 repetitions.\n3. **Endurance**: Track endurance by increasing the hold duration of each side (aim for 3-5 seconds at first and progress to 10 seconds or more) to enhance muscle stability and strength.\n\n### Alignment with Fitness Goals\n- **Str

Fitness Coach interacting with Older Individuals

In [37]:
user_message = f"""Analyze the given exercise using the information provided below. Provide clear and simple explanations of the exercise and its benefits. Emphasize key safety considerations, such as maintaining balance and using supports if necessary. Offer modifications to accommodate limited mobility or existing health conditions. Highlight the importance of exercises in improving strength, flexibility, and overall well-being, and encourage regular practice as part of a healthy lifestyle. Ensure that the guidance is accessible and encourages confidence and independence.

- User Profile: {User_profile}
- Exercise Being Done: {predicted_user_exercise}
- Grounded Description of the Exercise: {grounded_exercise_description}
"""

In [38]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a fitness coach working with older adults. Your priority is to ensure safety and simplicity in exercise instructions."},
    {"role": "user", "content": user_message}
  ]
)

In [39]:
message = response.choices[0].message.content

In [40]:
message

"**Exercise: Bird Dog**\n\n**Overview:**\nThe bird dog is an excellent exercise designed to improve core strength, balance, and stability. It primarily targets the abdominal muscles while also engaging the back, glutes, and shoulders. While it's categorized as an intermediate exercise, we can adapt it to make it safer and more accessible, especially for individuals like Elena, who may be managing pain from tennis elbow and needs to avoid strain on her arms.\n\n**How to Perform the Bird Dog:**\n\n1. **Start Position:**\n   - Begin on your hands and knees on a comfortable surface, like a yoga mat.\n   - Ensure your hands are directly under your shoulders, and your knees are under your hips. Keep your back flat, looking down at the mat to maintain a neutral spine.\n\n2. **Movement:**\n   - Slowly extend your right arm forward and your left leg backward at the same time. Make sure to keep both limbs at about hip height.\n   - Hold this position for a moment, focusing on your balance.\n   -